<a href="https://colab.research.google.com/github/its4ehk/AAT3020/blob/main/notebooks/1_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word2Vec Implementation from Scratch

This notebook demonstrates how to implement the Word2Vec algorithm from scratch using PyTorch. We'll use the first Harry Potter book as our corpus to train word embeddings.


## 1. Setting Up the Environment

First, we need to import the necessary libraries:
- `torch` and `torch.nn` for tensor operations and neural network functionality
- `string` for string manipulations (removing punctuation)


In [2]:
import torch
import torch.nn as nn
import string


## 2. Getting the Text Data

We'll download the first Harry Potter book to use as our corpus.

In [3]:
!wget "https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt"


--2025-03-20 04:46:31--  https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 439742 (429K) [text/plain]
Saving to: ‘J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt’

J. K. Rowling - Har 100%[===================>] 429.44K  --.-KB/s    in 0.05s   

2025-03-20 04:46:31 (9.03 MB/s) - ‘J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt’ saved [439742/439742]



## 3. Text Preprocessing

Before we can use the text data, we need to preprocess it:
- Remove punctuation
- Convert text to lowercase
- Split text into tokens (words)

This function will help us clean and tokenize the text.

In [4]:
def remove_punctuation(x):
  return x.translate(''.maketrans('', '', string.punctuation))

def make_tokenized_corpus(corpus):
  out= [ [y.lower() for y in remove_punctuation(sentence).split(' ') if y] for sentence in corpus]
  return [x for x in out if x!=[]]


## 4. Loading and Formatting the Text

Now we'll load the text file, replace some special characters, and split the text into sentences.


In [5]:
with open("J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt", 'r') as f:
  strings = f.readlines()
sample_text = "".join(strings).replace('\n', ' ').replace('Mr.', 'mr').replace('Mrs.', 'mrs').split('. ')


Let's tokenize the text using our preprocessing function `make_tokenized_corpus`:

In [6]:
# Corpus is a list of list of strings (words)

In [7]:
corpus = make_tokenized_corpus(sample_text)

corpus[:5]

[['harry',
  'potter',
  'and',
  'the',
  'sorcerers',
  'stone',
  'chapter',
  'one',
  'the',
  'boy',
  'who',
  'lived',
  'mr',
  'and',
  'mrs',
  'dursley',
  'of',
  'number',
  'four',
  'privet',
  'drive',
  'were',
  'proud',
  'to',
  'say',
  'that',
  'they',
  'were',
  'perfectly',
  'normal',
  'thank',
  'you',
  'very',
  'much'],
 ['they',
  'were',
  'the',
  'last',
  'people',
  'youd',
  'expect',
  'to',
  'be',
  'involved',
  'in',
  'anything',
  'strange',
  'or',
  'mysterious',
  'because',
  'they',
  'just',
  'didnt',
  'hold',
  'with',
  'such',
  'nonsense'],
 ['mr',
  'dursley',
  'was',
  'the',
  'director',
  'of',
  'a',
  'firm',
  'called',
  'grunnings',
  'which',
  'made',
  'drills'],
 ['he',
  'was',
  'a',
  'big',
  'beefy',
  'man',
  'with',
  'hardly',
  'any',
  'neck',
  'although',
  'he',
  'did',
  'have',
  'a',
  'very',
  'large',
  'mustache'],
 ['mrs',
  'dursley',
  'was',
  'thin',
  'and',
  'blonde',
  'and',
  'had

## 5. Creating Context Word Pairs

A key concept in Word2Vec is learning from context. We need to create pairs of words that appear near each other in the text. We'll use a sliding window approach to create these pairs.

For example, with the window size of 2, for the word "to" in the sentence "they were the last people youd expect to be involved...", we would create pairs with:
- ("to", "expect")
- ("to", "be")
- ("to", "involved")
- ("to", "in")

These pairs will be our training data.

In [8]:
from tqdm import tqdm

sample_sentence = ['they', 'were', 'the', 'last', 'people', 'youd', 'expect', 'to', 'be', 'involved', 'in', 'anything', 'strange', 'or', 'mysterious', 'because', 'they', 'just', 'didnt', 'hold', 'with', 'such', 'nonsense']

word_pairs = []
window_size = 2

for sample_sentence in tqdm(corpus):
  for cur_idx, center_word in enumerate(sample_sentence):
    window_begin = max(cur_idx - window_size, 0)
    window_end = min(cur_idx + window_size + 1, len(sample_sentence))
    # for context_word in sample_sentence[window_begin:window_end]:
    #   # if center_word == context_word: continue
    #   word_pairs.append( (center_word, context_word))
    for j in range(window_begin, window_end):
      if cur_idx == j: continue
      word_pairs.append( (center_word, sample_sentence[j]))

print(f"\nLength of word_pairs is {len(word_pairs)}")
print(f"First 5 example of word_pairs is {word_pairs[:5]}")

100%|██████████| 4682/4682 [00:00<00:00, 29486.57it/s]


Length of word_pairs is 282372
First 5 example of word_pairs is [('harry', 'potter'), ('harry', 'and'), ('potter', 'harry'), ('potter', 'and'), ('potter', 'the')]


## 6. Building the Vocabulary

To work with word vectors, we need to create a vocabulary that maps each unique word to an index. We'll also filter out rare words that appear less than a certain number of times in the corpus.

### 6.1 Collecting All Words

First, let's collect all words in our corpus:


In [9]:
# we have to make vocabulary
sentence = corpus[0]
entire_words = []

for sentence in corpus:
  for word in sentence:
    entire_words.append(word)




### 6.2 Finding Unique Words

Now, let's find the unique words in our corpus:

In [10]:
# we have to get the "unique" item among total words

unique_words = set(entire_words)
len(unique_words)


6038

### 6.3 Converting to a List and Sorting

We'll convert the set of unique words to a sorted list:

In [11]:
# vocab_set[0] # set is not subscriptable because it has no order

unique_words = sorted(list(unique_words))
unique_words[0]


'\the'

### 6.4 Filtering by Frequency

Now, let's filter out rare words that occur less than a specified number of times:
- We can use the `Counter` class from the `collections` module to count the frequency of each word in the corpus.
- Caution on `alist.sort()` will return `None`.

In [12]:
# how can we filter the vocab by its frequency?
filtered_vocab = None
# you can use word counter as dictionary
# In python dictionary, dict.keys() gives keys, and dict.values() give values,
# dict.items() give (key, value)

from collections import Counter
word_counter = Counter(entire_words)
word_counter.most_common(10)
word_counter['harry']

threshold = 5
filtered_vocab = []

for key, value in word_counter.items():
  if value > threshold:
    filtered_vocab.append(key)

filtered_vocab.sort()
filtered_vocab[:10]

['a',
 'able',
 'abou',
 'about',
 'above',
 'across',
 'added',
 'afford',
 'afraid',
 'after']

## 7. Filtering Word Pairs

Now that we have our filtered vocabulary, we need to filter our word pairs to only include words that are in our vocabulary:

In [13]:
# Filter the word_pairs using the vocab
# word_pairs, filtered_vocab
# word_pairs is a list of [word_a, word_b]

filtered_word_pairs = []
vocab_set = set(filtered_vocab)

for pair in tqdm(word_pairs):
  a, b = pair
  if a in vocab_set and b in vocab_set:
    filtered_word_pairs.append(pair)


100%|██████████| 282372/282372 [00:00<00:00, 1946749.96it/s]


In [14]:
# implement same algorithm with list comprehension

filtered_word_pairs = [pair for pair in word_pairs if pair[0] in vocab_set and pair[1] in vocab_set]
filtered_word_pairs[0]

('harry', 'potter')

In [15]:
len(filtered_word_pairs), len(word_pairs)

(226846, 282372)

## 8. Converting Words to Indices

For efficiency, we'll convert our words to indices according to their position in our vocabulary:

In [16]:
# convert word into index of vocab
filtered_vocab.index('harry')

527

This is inefficient because `list.index()` has to scan the list every time. Let's use a dictionary for faster lookups:

In [17]:
# we can make it faster
# use dictionary to find the index of string
word2idx = dict()
for idx, word in enumerate(filtered_vocab):
  word2idx[word] = idx
word2idx['harry']

527

Now, let's convert our word pairs to index pairs more efficiently:

In [18]:
index_pairs = [(word2idx[pair[0]], word2idx[pair[1]]) for pair in filtered_word_pairs]
index_pairs[0]

(527, 953)

In [19]:
# Why we don't need idx2tok?

filtered_vocab[527]

'harry'

## 9. Creating Initial Word Vectors

Now we'll create random vectors for each word in our vocabulary. These vectors will be adjusted during training:
- We can use `torch.randn` to create random vectors that follow normal distribution.

In [20]:
# we have to make random vectors for each word in the vocab
# we also have to decide the dimension of the vector

dim = 100
vocab_size = len(filtered_vocab)

word_vectors = torch.randn(vocab_size, dim) / 10
word_vectors

tensor([[ 0.0125, -0.0370, -0.0671,  ...,  0.0198, -0.0279,  0.0918],
        [ 0.1049,  0.0743,  0.0598,  ...,  0.2685,  0.0057,  0.0385],
        [-0.0101,  0.0855,  0.0920,  ...,  0.0360,  0.0085,  0.1022],
        ...,
        [-0.0077, -0.1131,  0.1762,  ...,  0.0134,  0.1058, -0.0752],
        [ 0.0978, -0.0145, -0.0401,  ...,  0.1495, -0.2357,  0.1673],
        [-0.0581,  0.1191,  0.0291,  ..., -0.0869,  0.1212,  0.0894]])

In [21]:
# what is the vector for harry?

word_vectors[word2idx['harry']]


tensor([ 0.0881, -0.0740, -0.0561, -0.1266,  0.1663, -0.0900,  0.0298,  0.0351,
         0.1492, -0.0421, -0.0466,  0.1441,  0.2179, -0.0794,  0.0028, -0.0967,
         0.0440, -0.0257, -0.0207, -0.0489,  0.0107,  0.1389, -0.0006,  0.0966,
         0.0843,  0.0082, -0.0047,  0.0521,  0.1083, -0.1749, -0.0258, -0.0597,
        -0.0433,  0.0165, -0.0004, -0.0764,  0.1586, -0.1006, -0.0894, -0.0663,
         0.0185, -0.1242, -0.0476,  0.2011, -0.1548, -0.1597,  0.0778, -0.1771,
         0.0484,  0.0512,  0.0447, -0.0294,  0.0595, -0.0282, -0.0755, -0.0997,
        -0.0241,  0.1902, -0.0935, -0.0224, -0.0387,  0.1644, -0.0845, -0.0395,
        -0.0663,  0.0648,  0.0547,  0.0847, -0.0382,  0.0851, -0.0809,  0.0153,
         0.0265, -0.0220, -0.0864,  0.0620, -0.0104, -0.0424, -0.0993, -0.1280,
         0.0431,  0.0799,  0.0244,  0.1304,  0.0051,  0.0577, -0.0211,  0.0904,
        -0.0234,  0.0676, -0.0988,  0.0928, -0.0723,  0.2078, -0.0628, -0.0143,
        -0.1006, -0.0194, -0.0167, -0.17

## 10. Understanding Word Relationships with Dot Products

The core of Word2Vec is using dot products to measure relationships between words. Let's explore this concept:

In [22]:
torch.set_printoptions(sci_mode=False) # Do this to avoid scientific notation


## Dot Product
- Assume we have two vectors $a$ and $b$.
  - $a = [a_1, a_2, a_3, a_4, ..., a_n]$
  - $b = [b_1, b_2, b_3, b_4, ..., b_n]$
- $a \cdot b$ = $\sum _{i=1}^n a_ib_i$  = $a_1b_1 + a_2b_2 + a_3b_3 + a_4b_4 + ... + a_nb_n$

Let's calculate the dot product between "harry" and "potter":


In [23]:
# calculate P(potter|harry)
harry = word_vectors[word2idx['harry']]
potter = word_vectors[word2idx['potter']]
dot_product_value_between_potter_harry = sum(harry * potter)
dot_product_value_between_potter_harry

tensor(0.0447)

In [24]:
# we can get the dot product value for every other words in the vocab
# to get  P(word | harry)
word_dot_dict = {}
for word in filtered_vocab:
  w_idx = word2idx[word]
  w_vector = word_vectors[w_idx]
  word_dot_dict[word] = sum(w_vector * harry)
word_dot_dict

{'a': tensor(-0.0004),
 'able': tensor(0.0153),
 'abou': tensor(0.0524),
 'about': tensor(-0.0920),
 'above': tensor(0.0096),
 'across': tensor(-0.0128),
 'added': tensor(-0.0567),
 'afford': tensor(0.0608),
 'afraid': tensor(0.0495),
 'after': tensor(-0.0409),
 'afternoon': tensor(0.0808),
 'again': tensor(0.0114),
 'against': tensor(-0.0107),
 'ages': tensor(-0.0202),
 'ago': tensor(-0.0034),
 'agreed': tensor(-0.1055),
 'ah': tensor(0.0491),
 'ahead': tensor(-0.0417),
 'air': tensor(-0.2322),
 'albus': tensor(0.0376),
 'alive': tensor(0.0824),
 'all': tensor(0.1443),
 'alley': tensor(-0.0106),
 'allowed': tensor(0.0832),
 'almost': tensor(0.0170),
 'alone': tensor(0.0378),
 'along': tensor(-0.1158),
 'already': tensor(0.0009),
 'also': tensor(-0.0746),
 'although': tensor(0.0428),
 'always': tensor(-0.0535),
 'am': tensor(-0.0102),
 'an': tensor(-0.0681),
 'and': tensor(0.0537),
 'angrily': tensor(-0.0911),
 'angry': tensor(-0.1330),
 'another': tensor(-0.0369),
 'answer': tensor(0.

Now, let's convert these dot products to probabilities using the softmax function:
- We have to convert our prediction into probability distribution to get P(word|harry) so that sum of [P(a|harry), ..., P(potter|harry), ... P(ron|harry), ... ] = 1
- current dot product value is any real number, sometimes called as logit
  - logit from logistic regression. Some values that are not yet converted to 0-1 or value before sigmoid function
  - every probability should be in range (0, 1) (greater than 0, smaller than 1)
  - this can be handled by taking exponential of dot product values, divided by total sum
  - This function is called **Softmax**

- Why we use exponential?
  - Because we want to make every probability in positive range while preserving the order


In [36]:
from math import exp
word_exp_dict = {}
for word, dot_value in word_dot_dict.items():
  exp_value = torch.exp(dot_value)
  word_exp_dict[word] = exp_value
word_exp_dict
word_prob_dict = None

In [37]:
# Get P(potter|harry)

## 13. Efficient Matrix Operations
![img](https://mkang32.github.io/images/python/khan_academy_matrix_product.png)

Instead of calculating dot products one by one, we can use matrix multiplication for efficiency:


In [27]:
# get dot product result for every word in the vocabulary

# first, make vector_of_harry into matrix format

# do matrix multiplication


Let's verify that our matrix multiplication gives the same result as individual dot products:

Now let's implement the complete softmax calculation using matrix operations:


In [28]:
# convert dot product result into exponential

In [29]:
# get the sum of exponential


In [30]:
# divide exponential value with sum

## 14. Creating a Probability Function

Let's create a function to calculate probabilities efficiently:

In [31]:
def get_probs(query_vectors, entire_vectors):
  return None

# get_probs(mat_of_harry, word_vectors)

## 15. Preparing for Training

Before training our Word2Vec model, we need to split our dataset into training and testing sets:

In [32]:
# Now we can train the word2vec

# Let's think about training pairs
index_pairs # this is our dataset. It's list of list of two integer
# two integer means a pair of neighboring words

# Training set and Test set
# To validate that our model can solve 'unseen' problems
# So we have to split the dataset before training.

# To randomly split the dataset, we will first shuffle the dataset

# random.shuffle(index_pairs) # this will shuffle the list items

[(527, 953),
 (527, 33),
 (953, 527),
 (953, 33),
 (953, 1266),
 (33, 527),
 (33, 953),
 (33, 1266),
 (33, 1154),
 (1266, 953),
 (1266, 33),
 (1266, 1154),
 (1266, 1197),
 (1154, 33),
 (1154, 1266),
 (1154, 1197),
 (1154, 181),
 (1197, 1266),
 (1197, 1154),
 (1197, 181),
 (1197, 872),
 (181, 1154),
 (181, 1197),
 (181, 872),
 (181, 1266),
 (872, 1197),
 (872, 181),
 (872, 1266),
 (872, 123),
 (1266, 181),
 (1266, 872),
 (1266, 123),
 (1266, 1434),
 (123, 872),
 (123, 1266),
 (123, 1434),
 (123, 715),
 (1434, 1266),
 (1434, 123),
 (1434, 715),
 (1434, 795),
 (715, 123),
 (715, 1434),
 (715, 795),
 (715, 33),
 (795, 1434),
 (795, 715),
 (795, 33),
 (795, 796),
 (33, 715),
 (33, 795),
 (33, 796),
 (33, 311),
 (796, 795),
 (796, 33),
 (796, 311),
 (796, 863),
 (311, 33),
 (311, 796),
 (311, 863),
 (311, 858),
 (863, 796),
 (863, 311),
 (863, 858),
 (863, 437),
 (858, 311),
 (858, 863),
 (858, 437),
 (858, 962),
 (437, 863),
 (437, 858),
 (437, 962),
 (437, 300),
 (962, 858),
 (962, 437),
 

In [33]:
len(train_set), len(test_set)

NameError: name 'train_set' is not defined

## 16. Training the Word2Vec Model

Now we'll train our Word2Vec model using batched gradient descent:

In [ ]:
# making batch from train_set
# Batch is a set of training samples, that are calculated together
# And also we update the model after one single batch

## 17. Evaluating the Training

Let's visualize the training loss to see if our model is learning:

In [ ]:
import matplotlib.pyplot as plt
plt.plot(loss_record)

## 18. Testing the Model

Now we'll test our model on the test set:

## 19. Exploring Learned Word Relationships

Let's explore what our model has learned by finding the words most closely related to "harry":

In [ ]:
# P(potter|harry)?
